In [ ]:
#Meghna Allamudi and Olga Scrivner 
#Code for creating Graph Databse Corpora in Neo4J 
#Run "match (n) return (n)" to get graph in Neo4J Browser 
#Run "match (n) detach delete (n)" to delete each graph before running new iteration so they do not get plotted on same graph
#One unique constraint used on arg_cat in first two iterations

In [1]:
pip install neo4j

In [2]:
import pandas as pd 
from neo4j import GraphDatabase
driverr=GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", "Neo4JCorpus"))
session=driverr.session()

In [3]:
df = pd.read_csv("all-persuasion-convo-data.csv")
df

,displayName,uid,text,createdAt,ArgCategory,Position,ConvoNum
0,JPAG9k,JPAG9kET5Oh5kNtNjXYPJrmj3ys1,"Hi, how are you?",2022-02-11T18:37:41.033Z,Opening remarks,Persuader,1
1,Jp2xrD,Jp2xrDbuJZXwFLMmwJpZVEtQ0kt1,can't complain. how are you?,2022-02-11T18:37:53.654Z,Opening remarks user response,Persuadee,1
2,JPAG9k,JPAG9kET5Oh5kNtNjXYPJrmj3ys1,Not bad either! Have you heard of online learn...,2022-02-11T18:38:01.824Z,Experience-related inquiry,Persuader,1
3,Jp2xrD,Jp2xrDbuJZXwFLMmwJpZVEtQ0kt1,yes and I've gone through it both in high scho...,2022-02-11T18:38:17.966Z,Experience-related user response,Persuadee,1
4,JPAG9k,JPAG9kET5Oh5kNtNjXYPJrmj3ys1,Interesting! It sounds like a pretty good idea...,2022-02-11T18:39:02.761Z,Logical appeal,Persuader,1
...,...,...,...,...,...,...,...
226,JPAG9kET5Oh5kNtNjXYPJrmj3ys1,JPAG9kET5Oh5kNtNjXYPJrmj3ys1,Have you experienced online learning?,2022-03-09T23:14:25.790Z,Experience-related inquiry,Persuadee,20
227,JPAG9kET5Oh5kNtNjXYPJrmj3ys1,JPAG9kET5Oh5kNtNjXYPJrmj3ys1,"If so, did you think being able to take classe...",2022-03-09T23:14:25.790Z,Thought-provoking inquiry,Persuadee,20
228,tb8fIF,tb8fIFT7vuXntLdbPMHG3wHitHH2,"Yeah, I've had online learning, I think that i...",2022-03-09T23:16:02.453Z,Experience-related inquiry user response,Persuadee,20
229,JPAG9kET5Oh5kNtNjXYPJrmj3ys1,JPAG9kET5Oh5kNtNjXYPJrmj3ys1,"I see, that definitely makes sense as learning...",2022-03-09T23:16:47.281Z,Closing remarks,Persuadee,20


In [5]:
#itration number one, just plotting persuader strategies and sequences for all conversations
#unique constraint was placed on arg_cat 
q1= """
MERGE (c1:ArgCategory {arg_cat:$arg_cat}) 
MERGE (c2:ArgCategory {arg_cat:$arg_cat_two}) 
CREATE (c1)-[:LEADS_TO]->(c2)
"""

convos = df["ConvoNum"]

for cnum in convos:
    groupingBySeparateConvo=df.groupby("ConvoNum")[["ArgCategory", "Position"]]
    groupRecieved=groupingBySeparateConvo.get_group(cnum)[["ArgCategory", "Position"]]
    persuaderArgs=groupRecieved.loc[df["Position"] == "Persuader", "ArgCategory"]
    newList = []
    for w in persuaderArgs:
        newList.append(w)
    index = 0
    while index < len(newList)-1:
        p={"arg_cat":newList[index],"arg_cat_two":newList[index+1]}
        index += 1
        session.run(q1,p)

In [6]:
#itration number two, plotting persuader and persuadee strategies and sequences for all conversations
#unique constraint was placed on arg_cat 
q1= """
MERGE (c1:ArgCategory {arg_cat:$arg_cat}) 
MERGE (c2:ArgCategory {arg_cat:$arg_cat_two}) 
CREATE (c1)-[:LEADS_TO]->(c2)
"""

convos = df["ConvoNum"]

for cnum in convos:
    groupingBySeparateConvo=df.groupby("ConvoNum")[["ArgCategory", "Position"]]
    groupRecieved=groupingBySeparateConvo.get_group(cnum)[["ArgCategory", "Position"]]
    #persuaderArgs=groupRecieved.loc[df["Position"] == "Persuader", "ArgCategory"]
    newList = []
    for w in groupRecieved.ArgCategory:
        newList.append(w)
    index = 0
    while index < len(newList)-1:
        p={"arg_cat":newList[index],"arg_cat_two":newList[index+1]}
        index += 1
        session.run(q1,p)

In [7]:
#itration number three, plotting each dialogue and sequence including argument category it belongs to
#no unique constraints used
q1= """
MERGE (t1:text {text:$textOne,arg_cat:$arg_cat})
MERGE (t2:text {text:$textTwo,arg_cat:$arg_cat_2})
MERGE (t1)-[:LEADS_TO]->(t2)
"""

convos = df["ConvoNum"];

for cnum in convos: 
    groupingBySeparateConvo=df.groupby("ConvoNum")[["ArgCategory", "text"]]
    groupRecieved=groupingBySeparateConvo.get_group(cnum)[["ArgCategory", "text"]]
    newListArg = []
    newListText = []
    for w in groupRecieved.ArgCategory:
        newListArg.append(w)
    for w in groupRecieved.text:
        newListText.append(w)
    index = 0
    while index < len(newListArg)-1:
        p={"arg_cat":newListArg[index], "arg_cat_2":newListArg[index+1], "textOne":newListText[index], "textTwo":newListText[index+1]}
        index += 1
        session.run(q1,p)
